In [ ]:
pip install torchvision
pip install opencv-python

In [ ]:
import torch
import numpy as np
import cv2

from torchvision import models, transforms
from PIL import Image

from week16.CAM_vis import CAM_localization
from week16.imagenet_class_map import imagenet_classmap

In [ ]:
resnet = models.resnet50(pretrained=True)
# Достаем обучающие параметры
params = list(resnet.parameters())
# и потом берем только веса последнего полносвязного слоя
weights_softmax = np.squeeze(params[-2].data.numpy())

# Forward модели
res50_model = models.resnet50(pretrained=True)

# Пишем callback, который сработает,
# когда хотим достать выход после нужного слоя
last_convs = []
def hook(module, input, output):
    last_convs.append(output)
        
res50_model.layer4[2].conv3.register_forward_hook(hook)

In [ ]:
def model_forward(batch_t):
    resnet = models.resnet50(pretrained=True)
    params = list(resnet.parameters())
    weights_softmax = np.squeeze(params[-2].data.numpy())

    last_convs = []
    def hook(module, input, output):
        last_convs.append(output)

    res50_model = models.resnet50(pretrained=True)
    res50_model.layer4[2].conv3.register_forward_hook(hook)

    out = res50_model(batch_t)
    class_idx = np.argmax(out[0].detach().numpy())
    print(class_idx, imagenet_classmap[class_idx])

    return class_idx, last_convs, weights_softmax